# Imports

In [1]:
from sklearn.svm import SVC
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.preprocessing import Normalizer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

# Data Prep (Training)


In [2]:
dataTrain = pd.read_csv('TrainDataset2024.csv', index_col=0)

#MISSING DATA
dataTrain.replace(999, np.nan, inplace=True)

imputer = SimpleImputer(strategy='median')
dataTrain = pd.DataFrame(imputer.fit_transform(dataTrain), columns=dataTrain.columns,index=dataTrain.index)


target = dataTrain[['pCR (outcome)']]#'RelapseFreeSurvival (outcome)']]
dataTrain.drop(columns=['pCR (outcome)','RelapseFreeSurvival (outcome)'], axis=1, inplace=True)

key_features = dataTrain[['ER', 'HER2', 'Gene']]
dataTrain.drop(columns=['ER', 'HER2', 'Gene'], axis=1, inplace=True)


In [3]:

#NORMALISATION
normalizer = Normalizer()
vector_normalized_data_train = normalizer.fit_transform(dataTrain)


In [4]:

#FEATURE REDUCTION
pca = PCA(n_components=0.95)
data_reduced_train = pca.fit_transform(vector_normalized_data_train)

pca_complete_train = pd.DataFrame(data_reduced_train, index=dataTrain.index)
pca_complete_train = pd.concat([pca_complete_train, key_features], axis=1)


# Data Prep (Test)


In [5]:
dataTest = pd.read_csv('FinalTestDataset2024.csv', index_col=0)

#MISSING DATA
dataTest.replace(999, np.nan, inplace=True)

imputer = SimpleImputer(strategy='median')
dataTest = pd.DataFrame(imputer.fit_transform(dataTest), columns=dataTest.columns,index=dataTest.index)

key_features = dataTest[['ER', 'HER2', 'Gene']]
dataTest.drop(columns=['ER', 'HER2', 'Gene'], axis=1, inplace=True)


In [6]:

#NORMALISATION
normalizer = Normalizer()
vector_normalized_data_test = normalizer.fit_transform(dataTest)


In [7]:

#FEATURE REDUCTION
data_reduced_test = pca.transform(vector_normalized_data_test)

pca_complete_test = pd.DataFrame(data_reduced_test, index=dataTest.index)
pca_complete_test = pd.concat([pca_complete_test, key_features], axis=1)


# Training

In [8]:
model = SVC(class_weight='balanced',kernel='rbf', C=1, gamma=0.01)

data = pca_complete_train.rename(str,axis="columns") 
    
# Split data into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=42)

# Train the model on the training data
model.fit(X_train, np.ravel(y_train))

# Predict on the test set
data = pca_complete_test.rename(str,axis="columns") 
predictions = model.predict(data)

results = pd.DataFrame({
    'Index': data.index,
    'Prediction': predictions
})

results.to_csv('PCRPrediction.csv', index=False)